In [1]:
import pandas as pd
import pickle
import json
from gensim.models import Word2Vec
from gensim.corpora import Dictionary
from gensim.models.callbacks import CallbackAny2Vec
from gensim.models import KeyedVectors
import numpy as np

In [2]:
metadata_df = pd.read_json("../data/metadata_rich_df.json")


In [3]:
sourcepath = "../data/large_files/trigrams_lemmatized/"
with open(sourcepath + "trigrams_lemmata_id_0.txt".format(str(id)), "r") as f:
    trigrams_list = f.readlines()

In [4]:

[el.strip() for el in trigrams_list][:10]

['religious matter',
 'divine hope',
 'divine hope',
 'leave unresolved',
 'leave unresolved',
 'racial equality',
 'racial equality',
 'joint action',
 'joint action',
 'joint action']

In [5]:
bidecades_strs = sorted([el for el in list(set(metadata_df["bidecade"])) if el != None])
bidecades_strs

['1900-1919', '1920-1939', '1940-1959', '1960-1979', '1980-1999', '2000-2019']

In [6]:
bidecade = "1900-1919"
ids = metadata_df[metadata_df["bidecade"]==bidecade]["id_kase"]
len(ids)

782

In [27]:
trigrams_lemmata = []
for id in ids:
    with open("../data/large_files/trigrams_lemmatized/trigrams_lemmata_id_{}.txt".format(str(id)), "r") as f:
        trigrams_lemmata.extend([el for el in f.readlines()])

In [28]:
trigrams_lemmata

['Michigan Ann Arbor\n',
 'Bonner University\n',
 'Bonner University\n',
 'Iowa Iowa\n',
 'Iowa Iowa\n',
 'Kansas Lawrence\n',
 'Ginn Co\n',
 'Elliott University\n',
 'Elliott University\n',
 'Kansas Lawrence Kansas\n',
 'Northfield Minnesota\n',
 'Drake University Des\n',
 'aspect athenian\n',
 'Paul epistle\n',
 'Paul epistle\n',
 'various paper\n',
 'various paper\n',
 'place assembly\n',
 'State College\n',
 'Hellems University\n',
 'Hellems University\n',
 'meeting Thursday\n',
 'meeting Thursday\n',
 'Boulder Colorado\n',
 'Word Order\n',
 'Barber University\n',
 'Barber University\n',
 'Missouri Columbia\n',
 'final copy\n',
 'final copy\n',
 'final copy\n',
 'Walker University\n',
 'Walker University\n',
 'University Minnesota\n',
 'Chicago Chicago\n',
 'Chicago Chicago\n',
 'Classical Journal\n',
 'Classical Journal\n',
 'Classical Journal\n',
 'Chamber Commerce\n',
 'Kelsey University\n',
 'Kelsey University\n',
 "o'clock Saturday\n",
 "o'clock Saturday\n",
 'State Normal Sch

In [7]:
for bidecade in bidecades_strs:
    ids = metadata_df[metadata_df["bidecade"]==bidecade]["id_kase"]
    trigrams_lemmata = []
    for id in ids:
        with open("../data/large_files/trigrams_lemmatized/trigrams_lemmata_id_{}.txt".format(str(id)), "r") as f:
            trigrams_lemmata.extend([el for el in f.readlines()])
    with open("../data/large_files/bidecade_trigrams_{}.txt".format(bidecade), "w") as f:
        f.writelines(trigrams_lemmata)
    print(bidecade, len(trigrams_lemmata))

1900-1919 722798
1920-1939 793794
1940-1959 1816044
1960-1979 3100450
1980-1999 8616167
2000-2019 45358562


# word2vec model development

In [14]:
class NgramCorpusSample(object):
    def __init__(self, filepath, sample_size=None, sample_seed=1, bow=False, dct=None):
        self.sample_size = sample_size
        with open(filepath, "r") as f:
            self.file_data = [el.strip() for el in f.readlines()]
        if sample_size != None:
            np.random.seed(sample_seed)
            sample_lines = np.random.randint(0, len(self.file_data), sample_size)
        else:
            sample_lines = np.array([n for n in range(0, self.file_data)])
        self.len = len(sample_lines)
        self.sample_lines = sample_lines
    def __len__(self):
        #[el for el in self]
        return self.len
    def __iter__(self):
        for li in self.sample_lines:
            line = self.file_data[li] # for line index in subcorpus lineindexes
            if bool(line.split()):
                yield line.split()

In [24]:
bidecade = "1900-1919"
sample_size = 1000000
subcorpus_ngrams =  NgramCorpusSample("../data/large_files/bidecade_trigrams_{}.txt".format(bidecade), sample_size=sample_size)

In [25]:
[doc for doc in subcorpus_ngrams][:10]

[['Historical', 'Publications'],
 ['die', 'Lxx'],
 ['external', 'evidence'],
 ['Lehre', 'von', 'der'],
 ['mean', 'death'],
 ['perfect', 'right'],
 ['Theologische', 'Studien', 'und'],
 ['new', 'order'],
 ['World', 'God'],
 ['Paul', 'merge']]

In [26]:
bidecades_vocabs_counts_df = pickle.load(open("../data/bidecades_vocabs_counts_df.pickle", "rb"))
bidecades_vocabs_counts_df.head(5)

,1900-1919,1920-1939,1940-1959,1960-1979,1980-1999,2000-2019
man,11218.0,8342.0,24108.0,39199.0,48435.0,100670.0
new,10183.0,10192.0,23469.0,42092.0,97421.0,293457.0
Jesus,9650.0,4911.0,12134.0,23416.0,49765.0,215687.0
et,9325.0,11818.0,20805.0,35969.0,49326.0,73464.0
God,9248.0,6276.0,26328.0,45751.0,99505.0,293240.0


In [27]:
model = Word2Vec(vector_size=100, window=3, negative=5, ns_exponent=1, sg=0, workers=8)
freqs_tups = dict(bidecades_vocabs_counts_df[bidecade]).items()
model.build_vocab_from_freq(word_freq=dict(freqs_tups))

In [28]:
%%time
model.train(corpus_iterable=subcorpus_ngrams, total_examples=sample_size, epochs=5)

CPU times: user 40.7 s, sys: 3.19 s, total: 43.9 s
Wall time: 38.8 s


(6931103, 10293575)

In [29]:
model.wv.most_similar("Paul")

[('pastoral', 0.6063287258148193),
 ('clement', 0.6025952696800232),
 ('Augustine', 0.6009352207183838),
 ('Creed', 0.5917514562606812),
 ('pillar', 0.5720595717430115),
 ('Philo', 0.5548839569091797),
 ('James', 0.5362641215324402),
 ('Peter', 0.5349253416061401),
 ('Newman', 0.5310553312301636),
 ('Pauline', 0.518301248550415)]

In [32]:
sample_size = 1000000
for bidecade in bidecades_strs:
    subcorpus_ngrams =  NgramCorpusSample("../data/large_files/bidecade_trigrams_{}.txt".format(bidecade), sample_size=sample_size)
    model = Word2Vec(vector_size=100, window=3, negative=5, ns_exponent=1, sg=0, workers=8)
    freqs_tups = dict(bidecades_vocabs_counts_df[bidecade])
    model.build_vocab_from_freq(word_freq=freqs_tups)
    print("going to train " + bidecade + " subcorpus model")
    model.train(corpus_iterable=subcorpus_ngrams, total_examples=sample_size, epochs=5)
    vectors = model.wv
    vectors.save("../data/large_files/vectors_{0}_sample{1}.kv".format(bidecade, str(sample_size)))
    print("finished training " + bidecade + " subcorpus model")

going to train 1900-1919 subcorpus model
finished training 1900-1919 subcorpus model
going to train 1920-1939 subcorpus model
finished training 1920-1939 subcorpus model
going to train 1940-1959 subcorpus model
finished training 1940-1959 subcorpus model
going to train 1960-1979 subcorpus model
finished training 1960-1979 subcorpus model
going to train 1980-1999 subcorpus model
finished training 1980-1999 subcorpus model
going to train 2000-2019 subcorpus model
finished training 2000-2019 subcorpus model


# Backup

In [113]:
model.wv.most_similar("Jesus")

[('crucified', 0.6054939031600952),
 ('risen', 0.5864917039871216),
 ('Ali', 0.5596235394477844),
 ('parousia', 0.5448322892189026),
 ('supper', 0.5399717688560486),
 ('crucify', 0.539387583732605),
 ('Philem', 0.5347006916999817),
 ('Grillmeier', 0.5329602360725403),
 ('Epaphras', 0.5325378179550171),
 ('Nicodemus', 0.5318003296852112)]

In [115]:
len(model.wv.index_to_key)

41385

In [117]:
words_to_keep = [tup[0] for tup in freqs_tups[:3000]]

In [118]:
def filter_wv(wv, words_to_keep):
    model_words = wv.index_to_key
    words_to_trim = list((set(model_words).difference(set(words_to_keep))))
    ids_to_trim = [wv.key_to_index[w] for w in words_to_trim]
    for w in words_to_trim:
        del wv.key_to_index[w]
    wv.vectors = np.delete(wv.vectors, ids_to_trim, axis=0)
    wv.fill_norms()
    for i in sorted(ids_to_trim, reverse=True):
        del(wv.index_to_key[i])
    return wv

In [119]:
test_wv = model.wv

In [120]:
model_vw_filtered = filter_wv(test_wv, words_to_keep)

In [ ]:
# but there is much more straightforward solution to this!
model_vw_filtered = model.wv.vectors_for_all(words_to_keep)

In [121]:
len(model_vw_filtered)

3000

In [124]:
model_vw_filtered.most_similar("Paul")

[('Pauline', 0.807790994644165),
 ('colossian', 0.4637281894683838),
 ('athanasius', 0.43054693937301636),
 ('contributor', 0.4175826907157898),
 ('ephesian', 0.41415196657180786),
 ('inspire', 0.4122663140296936),
 ('Titus', 0.40873152017593384),
 ('shape', 0.4032384753227234),
 ('Robinson', 0.3954716622829437),
 ('Hebrews', 0.39504387974739075)]

In [134]:
def ids_from_colvals(df_name, col, matchstring):
    ids = eval('{0}[{0}["{1}"]{2}]'.format(df_name, col, matchstring))["id"].tolist()
    return ids

In [137]:
generations = []
for n in range (1900, 2001, 20):
    generations.append((str(n) + "," + str(n+19)))
generations

['1900,1919', '1920,1939', '1940,1959', '1960,1979', '1980,1999', '2000,2019']

In [139]:
test_gen = generations[-2]
test_gen

'1980,1999'

In [140]:
ids = ids_from_colvals("jstor_df", "publicationYear", ".between({})".format(test_gen))
len(ids)

2520

In [141]:
corpus = FriendlyCorpus()
corpus_len = len([doc for doc in corpus])
model = Word2Vec(vector_size=150, window=3, negative=5, ns_exponent=1, sg=0, workers=8)
model.build_vocab_from_freq(word_freq=dict(freqs_tups))
model.train(corpus_iterable=corpus, total_examples=corpus_len, epochs=5)
model_vectors = model.wv

(39413004, 40391550)

In [149]:
model_vectors.save("../data/large_files/model_vectors_{}.wv".format(test_gen))

In [154]:
model_vectors_loaded = KeyedVectors.load("../data/large_files/model_vectors_{}.wv".format(test_gen))

# word2vec main application

In [157]:
generations = []
for n in range (1900, 2001, 20):
    generations.append((str(n) + "," + str(n+19)))
generations

['1900,1919', '1920,1939', '1940,1959', '1960,1979', '1980,1999', '2000,2019']

In [158]:
for gen in generations:
    print("generation {} model training".format(gen))
    ids = ids_from_colvals("jstor_df", "publicationYear", ".between({})".format(test_gen))
    corpus = FriendlyCorpus()
    corpus_len = len([doc for doc in corpus])
    model = Word2Vec(vector_size=150, window=3, negative=5, ns_exponent=1, sg=0, workers=8)
    model.build_vocab_from_freq(word_freq=dict(freqs_tups))
    model.train(corpus_iterable=corpus, total_examples=corpus_len, epochs=5)
    model_vectors = model.wv
    print("generation {} ready. saving...".format(gen))
    model_vectors.save("../data/large_files/model_vectors_{}.wv".format(gen))

generation 1900,1919 model training
generation 1900,1919 ready. saving...
generation 1920,1939 model training
generation 1920,1939 ready. saving...
generation 1940,1959 model training
generation 1940,1959 ready. saving...
generation 1960,1979 model training
generation 1960,1979 ready. saving...
generation 1980,1999 model training
generation 1980,1999 ready. saving...
generation 2000,2019 model training
generation 2000,2019 ready. saving...
